# SmartStyle Algorithm Coding

## In the following code, the SmartStyle Algorithm is built from scratch using R. A mock dataset was created and imported into R, showing data that might be generated from TikTok videos styling multiple Aritzia pieces together, as well as from website analytics showing which items are commonly purchased together to form an outfit. 

## The knn classifier learns from the mock data we present to it, and shows a 82% accuracy in predicting the correct item to complete the outfit. For example, given the Ganna Jacket, the Melina Pant and The Contour Squareneck Bodysuit, the algorithm will predict that the New Balance 2002R Shoes sold at Aritzia would pair well with this combination of items.

## The following code runs through the generation of the algorithm, and then tests the algorithm to provide four different outfit suggestions at the bottom of the page. Headers section the code into topics and code comments narrate what each cell is doing.

In [ ]:
# Here I am installing all packages needed for my code to run.
install.packages("tidymodels")
install.packages("tidyverse")
install.packages("repr")
install.packages("caret")
install.packages("kknn")

In [1]:
# Here I am loading my packages.
library(tidyverse)
library(repr)
library(tidymodels)
library(kknn)
library(caret)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.3     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ 

### Mock Dataset:

In [2]:
# Reading mock data from Tiktok and previous sales analytics into R.
Aritzia_data <- read_csv("data/Artizia.csv") 
Aritzia_data

Rows: 99 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Top, Bottom, Shoes, Jacket

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Top,Bottom,Shoes,Jacket
<chr>,<chr>,<chr>,<chr>
Contour_Bodysuit_Squareneck,Melina_Pant,New_Balance_2002R,Ganna_Jacket
Tna_Karealis_Tank,Cozy_Fleece_Boyfriend_Sweatpant,Tna_X_Emu_Stinger_Micro,Cozy_Fleece_Perfect_Zip_Hoodie
Contour_Bodysuit_Scoopneck,Babaton_Carrot_Pant,Vans_Classic_Slip-On_98_Dx,Babaton_Agency_Blazer
Everyday_TShirt,Tna_Butter_Legging,Salomon_Xa_Pro_1_Mid,Super_Puff_Shorty
Contour_Bodysuit_Squareneck,Melina_Pant,New_Balance_2002R,Ganna_Jacket
Tna_Karealis_Tank,Cozy_Fleece_Boyfriend_Sweatpant,Tna_X_Emu_Stinger_Micro,Cozy_Fleece_Perfect_Zip_Hoodie
Contour_Bodysuit_Scoopneck,Babaton_Carrot_Pant,Vans_Classic_Slip-On_98_Dx,Babaton_Agency_Blazer
Everyday_TShirt,Tna_Butter_Legging,Salomon_Xa_Pro_1_Mid,Super_Puff_Shorty
Contour_Bodysuit_Squareneck,Melina_Pant,New_Balance_2002R,Ganna_Jacket


# Building the SmartStyle Outfit Prediction Model:

## Classification model to predict the type of top to wear given a bottom, shoes and jacket.

In [38]:
#Here, I am breaking down the above dataset into more columns that can be more easily interpreted by my algorithm:
#One-hot-encoding categorical features:
#define one-hot encoding function
dummy <- dummyVars(" ~ . -Top", data= Aritzia_data)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=Aritzia_data))

#view final data frame and merge "Top" column to final dataset:
library(dplyr)
final_df <- bind_cols(final_df, Aritzia_data['Top'])
final_df2 <- final_df %>% 
  mutate(Top = recode(Top, Contour_Bodysuit_Squareneck = '1', Tna_Karealis_Tank = '2', Contour_Bodysuit_Scoopneck =  '3',Everyday_TShirt = '4'))

final_df2$Top = as.factor(as.character(final_df2$Top))
final_df2

,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Top
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,0,1,0,1,0,0,0,0,0,1,0,1
2,0,1,0,0,0,0,1,0,0,1,0,0,2
3,1,0,0,0,0,0,0,1,1,0,0,0,3
4,0,0,0,1,0,1,0,0,0,0,0,1,4
5,0,0,1,0,1,0,0,0,0,0,1,0,1
6,0,1,0,0,0,0,1,0,0,1,0,0,2
7,1,0,0,0,0,0,0,1,1,0,0,0,3
8,0,0,0,1,0,1,0,0,0,0,0,1,4
9,0,0,1,0,1,0,0,0,0,0,1,0,1


In [4]:
#Here I am splitting my data into testing and training data:
set.seed(1)
   aritzia_split <- initial_split(final_df2, prop = 0.75, strata = Top)
   aritzia_train <- training(aritzia_split)
   aritzia_test <- testing(aritzia_split)

In [5]:
#Here I am setting the specifications of my model:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 12) |> 
set_engine("kknn") |>
       set_mode("classification")

In [6]:
#Here I am creating a recipe for my model:
aritzia_recipe <- recipe(Top ~ ., data = aritzia_train)

In [8]:
# fitting the classification model onto the training data
aritzia_fit <- workflow() |> 
add_recipe(aritzia_recipe) |>
add_model(knn_spec) |> 
fit(data = aritzia_train)

In [9]:
#fitting the model onto testing data
aritzia_test_predictions <- predict(aritzia_fit, aritzia_test) |>
  bind_cols(aritzia_test)
aritzia_test_predictions

#computing accuracy metrics for the classification model
accuracy <- aritzia_test_predictions |>
  metrics(truth = Top, estimate = .pred_class)

accuracy2 <- aritzia_test_predictions |>
  conf_mat(truth = Top, estimate = .pred_class)

,.pred_class,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Top
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,2,0,1,0,0,0,0,1,0,0,1,0,0,2
7,3,1,0,0,0,0,0,0,1,1,0,0,0,3
10,2,0,1,0,0,0,0,1,0,0,1,0,0,2
12,4,0,0,0,1,0,1,0,0,0,0,0,1,4
15,3,1,0,0,0,0,0,0,1,1,0,0,0,3
18,2,0,1,0,0,0,0,1,0,0,1,0,0,4
19,3,1,0,0,0,0,0,0,1,1,0,0,0,3
21,1,0,0,1,0,1,0,0,0,0,0,1,0,1
26,2,0,1,0,0,0,0,1,0,0,1,0,0,2


In [10]:
#accuracy metrics for classification model
accuracy
accuracy2

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.7777778
kap,multiclass,0.7038391


          Truth
Prediction 1 2 3 4
         1 4 0 1 0
         2 0 6 0 1
         3 1 0 6 3
         4 0 0 0 5

## Classification model to predict the type of bottom to wear given a top, shoes and jacket.

In [11]:
#Here, I am breaking down the above dataset into more columns that can be more easily interpreted by my algorithm:
#define one-hot encoding function
dummy2 <- dummyVars(" ~ . -Bottom", data= Aritzia_data)

#perform one-hot encoding on data frame
final_df3 <- data.frame(predict(dummy2, newdata=Aritzia_data))

#view final data frame and merge Bottom column to final dataset
library(dplyr)
final_df3 <- bind_cols(final_df3, Aritzia_data['Bottom'])
final_df3 <- final_df3 %>% 
  mutate(Bottom = recode(Bottom, Melina_Pant = '1', Cozy_Fleece_Boyfriend_Sweatpant = '2', Babaton_Carrot_Pant =  '3', Tna_Butter_Legging = '4'))

final_df3$Bottom = as.factor(as.character(final_df3$Bottom))
final_df3

,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Bottom
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,1,0,0,1,0,0,0,0,0,1,0,1
2,0,0,0,1,0,0,1,0,0,1,0,0,2
3,1,0,0,0,0,0,0,1,1,0,0,0,3
4,0,0,1,0,0,1,0,0,0,0,0,1,4
5,0,1,0,0,1,0,0,0,0,0,1,0,1
6,0,0,0,1,0,0,1,0,0,1,0,0,2
7,1,0,0,0,0,0,0,1,1,0,0,0,3
8,0,0,1,0,0,1,0,0,0,0,0,1,4
9,0,1,0,0,1,0,0,0,0,0,1,0,1


In [12]:
# Here I am splitting my testing and training data:
set.seed(1)
   aritzia_split3 <- initial_split(final_df3, prop = 0.75, strata = Bottom)
   aritzia_train3 <- training(aritzia_split3)
   aritzia_test3 <- testing(aritzia_split3)

In [13]:
# Here I am setting my model specifications:
knn_spec3 <- nearest_neighbor(weight_func = "rectangular", neighbors = 12) |> 
set_engine("kknn") |>
       set_mode("classification")

In [14]:
# Here I am setting my recipe:
aritzia_recipe3 <- recipe(Bottom ~ ., data = aritzia_train3)

In [15]:
# fitting the classification model onto the training data
aritzia_fit3 <- workflow() |> 
add_recipe(aritzia_recipe3) |>
add_model(knn_spec3) |> 
fit(data = aritzia_train3)

In [16]:
#fitting the model onto testing data
aritzia_test_predictions3 <- predict(aritzia_fit3, aritzia_test3) |>
  bind_cols(aritzia_test3)
aritzia_test_predictions3

#computing accuracy metrics for the classification model
accuracy3 <- aritzia_test_predictions3 |>
  metrics(truth = Bottom, estimate = .pred_class)

accuracy32 <- aritzia_test_predictions3 |>
  conf_mat(truth = Bottom, estimate = .pred_class)

,.pred_class,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Bottom
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,4,0,0,1,0,0,1,0,0,0,0,0,1,4
8,4,0,0,1,0,0,1,0,0,0,0,0,1,4
9,1,0,1,0,0,1,0,0,0,0,0,1,0,1
12,4,0,0,1,0,0,1,0,0,0,0,0,1,4
15,3,1,0,0,0,0,0,0,1,1,0,0,0,3
19,3,1,0,0,0,0,0,0,1,1,0,0,0,3
26,2,0,0,0,1,0,0,1,0,0,1,0,0,2
29,1,0,1,0,0,1,0,0,0,0,0,1,0,1
34,4,0,0,1,0,0,1,0,0,1,0,0,0,3


In [17]:
#accuracy metrics for classification model
accuracy3
accuracy32

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.7777778
kap,multiclass,0.7022059


          Truth
Prediction 1 2 3 4
         1 5 0 0 0
         2 0 4 0 0
         3 2 0 5 0
         4 0 2 2 7

## Classification model to predict the type of shoes to wear given a top, bottom and jacket.

In [18]:
#Here, I am breaking down the above dataset into more columns that can be more easily interpreted by my algorithm:
#define one-hot encoding function
dummy3 <- dummyVars(" ~ . -Shoes", data= Aritzia_data)

#perform one-hot encoding on data frame
final_df4 <- data.frame(predict(dummy3, newdata=Aritzia_data))

#view final data frame and merge Shoes column to final dataset
library(dplyr)
final_df4 <- bind_cols(final_df4, Aritzia_data['Shoes'])
final_df4 <- final_df4 %>% 
  mutate(Shoes = recode(Shoes, New_Balance_2002R = '1', Tna_X_Emu_Stinger_Micro = '2', "Vans_Classic_Slip-On_98_Dx" =  '3', Salomon_Xa_Pro_1_Mid = '4'))

final_df4$Shoes = as.factor(as.character(final_df4$Shoes))
final_df4

,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Shoes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,1,0,0,0,0,1,0,0,0,1,0,1
2,0,0,0,1,0,1,0,0,0,1,0,0,2
3,1,0,0,0,1,0,0,0,1,0,0,0,3
4,0,0,1,0,0,0,0,1,0,0,0,1,4
5,0,1,0,0,0,0,1,0,0,0,1,0,1
6,0,0,0,1,0,1,0,0,0,1,0,0,2
7,1,0,0,0,1,0,0,0,1,0,0,0,3
8,0,0,1,0,0,0,0,1,0,0,0,1,4
9,0,1,0,0,0,0,1,0,0,0,1,0,1


In [19]:
# Here I am splitting my testing and training data:
set.seed(1)
   aritzia_split4 <- initial_split(final_df4, prop = 0.75, strata = Shoes)
   aritzia_train4 <- training(aritzia_split4)
   aritzia_test4 <- testing(aritzia_split4)

In [20]:
# Here I am setting my model specifications:
knn_spec4 <- nearest_neighbor(weight_func = "rectangular", neighbors = 12) |> 
set_engine("kknn") |>
       set_mode("classification")

In [21]:
# Here I am setting a recipe for my model:
aritzia_recipe4 <- recipe(Shoes ~ ., data = aritzia_train4)

In [22]:
# fitting the classification model onto the training data
aritzia_fit4 <- workflow() |> 
add_recipe(aritzia_recipe4) |>
add_model(knn_spec4) |> 
fit(data = aritzia_train4)

In [23]:
#fitting the model onto testing data
aritzia_test_predictions4 <- predict(aritzia_fit4, aritzia_test4) |>
  bind_cols(aritzia_test4)
aritzia_test_predictions4

#computing accuracy metrics for the classification model
accuracy4 <- aritzia_test_predictions4 |>
  metrics(truth = Shoes, estimate = .pred_class)

accuracy42 <- aritzia_test_predictions4 |>
  conf_mat(truth = Shoes, estimate = .pred_class)

,.pred_class,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,JacketBabaton_Agency_Blazer,JacketCozy_Fleece_Perfect_Zip_Hoodie,JacketGanna_Jacket,JacketSuper_Puff_Shorty,Shoes
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,4,0,0,1,0,0,0,0,1,0,0,0,1,4
8,4,0,0,1,0,0,0,0,1,0,0,0,1,4
9,1,0,1,0,0,0,0,1,0,0,0,1,0,1
12,4,0,0,1,0,0,0,0,1,0,0,0,1,4
15,3,1,0,0,0,1,0,0,0,1,0,0,0,3
19,3,1,0,0,0,1,0,0,0,1,0,0,0,3
29,1,0,1,0,0,0,0,1,0,0,0,1,0,1
30,2,0,0,0,1,0,1,0,0,0,1,0,0,2
34,3,0,0,1,0,1,0,0,0,1,0,0,0,4


In [24]:
#accuracy metrics for classification model
accuracy4
accuracy42

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.7692308
kap,multiclass,0.6904762


          Truth
Prediction 1 2 3 4
         1 5 0 1 0
         2 0 4 0 0
         3 1 0 5 1
         4 0 2 1 6

## Classification model to predict the type of jacket to wear given a top, bottom and shoes.

In [25]:
#Here, I am breaking down the above dataset into more columns that can be more easily interpreted by my algorithm:
#define one-hot encoding function
dummy4 <- dummyVars(" ~ . -Jacket", data= Aritzia_data)

#perform one-hot encoding on data frame
final_df5 <- data.frame(predict(dummy4, newdata=Aritzia_data))

#view final data frame and merge Jacket column to final dataset
library(dplyr)
final_df5 <- bind_cols(final_df5, Aritzia_data['Jacket'])
final_df5 <- final_df5 %>% 
  mutate(Jacket = recode(Jacket, Ganna_Jacket = '1', Cozy_Fleece_Perfect_Zip_Hoodie = '2', Babaton_Agency_Blazer =  '3', Super_Puff_Shorty = '4'))

final_df5$Jacket = as.factor(as.character(final_df5$Jacket))
final_df5

,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,Jacket
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,1,0,0,0,0,1,0,1,0,0,0,1
2,0,0,0,1,0,1,0,0,0,0,1,0,2
3,1,0,0,0,1,0,0,0,0,0,0,1,3
4,0,0,1,0,0,0,0,1,0,1,0,0,4
5,0,1,0,0,0,0,1,0,1,0,0,0,1
6,0,0,0,1,0,1,0,0,0,0,1,0,2
7,1,0,0,0,1,0,0,0,0,0,0,1,3
8,0,0,1,0,0,0,0,1,0,1,0,0,4
9,0,1,0,0,0,0,1,0,1,0,0,0,1


In [26]:
# Here I am splitting my testing and training data:
set.seed(1)
   aritzia_split5 <- initial_split(final_df5, prop = 0.75, strata = Jacket)
   aritzia_train5 <- training(aritzia_split5)
   aritzia_test5 <- testing(aritzia_split5)

In [27]:
# Here I am setting my model specifications:
knn_spec5 <- nearest_neighbor(weight_func = "rectangular", neighbors = 12) |> 
set_engine("kknn") |>
       set_mode("classification")

In [28]:
#Here I am setting a recipe for my model:
aritzia_recipe5 <- recipe(Jacket ~ ., data = aritzia_train5)

In [29]:
# fitting the classification model onto the training data
aritzia_fit5 <- workflow() |> 
add_recipe(aritzia_recipe5) |>
add_model(knn_spec5) |> 
fit(data = aritzia_train5)

In [30]:
#fitting the model onto testing data
aritzia_test_predictions5 <- predict(aritzia_fit5, aritzia_test5) |>
  bind_cols(aritzia_test5)
aritzia_test_predictions5

#computing accuracy metrics for the classification model
accuracy5 <- aritzia_test_predictions5 |>
  metrics(truth = Jacket, estimate = .pred_class)

accuracy52 <- aritzia_test_predictions5 |>
  conf_mat(truth = Jacket, estimate = .pred_class)


,.pred_class,TopContour_Bodysuit_Scoopneck,TopContour_Bodysuit_Squareneck,TopEveryday_TShirt,TopTna_Karealis_Tank,BottomBabaton_Carrot_Pant,BottomCozy_Fleece_Boyfriend_Sweatpant,BottomMelina_Pant,BottomTna_Butter_Legging,ShoesNew_Balance_2002R,ShoesSalomon_Xa_Pro_1_Mid,ShoesTna_X_Emu_Stinger_Micro,ShoesVans_Classic_Slip.On_98_Dx,Jacket
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,4,0,0,1,0,0,0,0,1,0,1,0,0,4
8,4,0,0,1,0,0,0,0,1,0,1,0,0,4
9,1,0,1,0,0,0,0,1,0,1,0,0,0,1
12,4,0,0,1,0,0,0,0,1,0,1,0,0,4
15,3,1,0,0,0,1,0,0,0,0,0,0,1,3
19,3,1,0,0,0,1,0,0,0,0,0,0,1,3
29,1,0,1,0,0,0,0,1,0,1,0,0,0,1
30,2,0,0,0,1,0,1,0,0,0,0,1,0,2
33,3,0,0,1,0,0,0,1,0,0,1,0,0,3


In [31]:
#accuracy metrics for classification model
accuracy5
accuracy52

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.9615385
kap,multiclass,0.9484127


          Truth
Prediction 1 2 3 4
         1 6 0 0 0
         2 0 5 0 0
         3 1 0 7 0
         4 0 0 0 7

# Evauating the Prediction Accuracy of our model:

In [33]:
# Here I am computing the overall accuracy of the prediction model:
average_accuracy <- (0.9615385 + 0.7692308 + 0.7777778 + 0.7777778)/4
average_accuracy
Average_accuracy <- 82.16
Average_accuracy

[1] 0.8215812

[1] 82.16

# The average prediction accuracy of our SmartStyle algorithm for this mock dataset is 82%!



# Using our models to suggest clothing items to complete an outfit:

In [34]:
#Suggesting a top to complete the outfit
new_obs1 <- tibble(BottomBabaton_Carrot_Pant = 0, BottomCozy_Fleece_Boyfriend_Sweatpant=0,BottomMelina_Pant=1, BottomTna_Butter_Legging=0, ShoesNew_Balance_2002R =1, ShoesSalomon_Xa_Pro_1_Mid=0,ShoesTna_X_Emu_Stinger_Micro=0,ShoesVans_Classic_Slip.On_98_Dx=0,JacketBabaton_Agency_Blazer=0,JacketCozy_Fleece_Perfect_Zip_Hoodie=0,JacketGanna_Jacket=1,JacketSuper_Puff_Shorty=0)
predict(aritzia_fit, new_obs1)

.pred_class
<fct>
1


### Here, our model predicts that we should wear the Contour Bodysuit with a Squareneck with the Melina pant, Ganna Jacket and NB2002R's! This is exactly what we expected it to predict, as the dataset we manually created had that item combination manually inserted most frequently.

In [35]:
#Suggesting a bottom to complete an outfit
new_obs2 <- tibble(TopContour_Bodysuit_Scoopneck=0, TopContour_Bodysuit_Squareneck=0, TopEveryday_TShirt=0, TopTna_Karealis_Tank=1, ShoesNew_Balance_2002R =0, ShoesSalomon_Xa_Pro_1_Mid=0,ShoesTna_X_Emu_Stinger_Micro=1,ShoesVans_Classic_Slip.On_98_Dx=0,JacketBabaton_Agency_Blazer=0,JacketCozy_Fleece_Perfect_Zip_Hoodie=1,JacketGanna_Jacket=0,JacketSuper_Puff_Shorty=0)
predict(aritzia_fit3, new_obs2)

.pred_class
<fct>
2


### Here, our model predicts that we should wear the Cozy Boyfiend Sweatpant with the Karealis Tank, Boyfriend Zipup and Emus! This is exactly what we expected it to predict, as the dataset we manually created had that item combination manually inserted most frequently.

In [36]:
#Suggesting shoes to complete an outfit
new_obs3 <- tibble(BottomBabaton_Carrot_Pant=1, BottomCozy_Fleece_Boyfriend_Sweatpant=0, BottomMelina_Pant=0, BottomTna_Butter_Legging=0, TopContour_Bodysuit_Scoopneck=1, TopContour_Bodysuit_Squareneck=0, TopEveryday_TShirt=0, TopTna_Karealis_Tank=0, JacketBabaton_Agency_Blazer=1,JacketCozy_Fleece_Perfect_Zip_Hoodie=0,JacketGanna_Jacket=0,JacketSuper_Puff_Shorty=0)
predict(aritzia_fit4, new_obs3)

.pred_class
<fct>
3


### Here, our model predicts that we should wear the Vans Classic Slip Ons with the Agency blazer, Scoopneck Contour bodysuit and carrot pant! This is exactly what we expected it to predict, as the dataset we manually created had that item combination manually inserted most frequently.

In [37]:
#Suggesting a jacket to complete an outfit
new_obs4 <- tibble(ShoesNew_Balance_2002R=0, ShoesSalomon_Xa_Pro_1_Mid=1, ShoesTna_X_Emu_Stinger_Micro=0, ShoesVans_Classic_Slip.On_98_Dx=0, BottomBabaton_Carrot_Pant=0, BottomCozy_Fleece_Boyfriend_Sweatpant=0, BottomMelina_Pant=0, BottomTna_Butter_Legging=1, TopContour_Bodysuit_Scoopneck=0, TopContour_Bodysuit_Squareneck=0, TopEveryday_TShirt=1, TopTna_Karealis_Tank=0)
predict(aritzia_fit5, new_obs4)

.pred_class
<fct>
4


### Here, our model predicts that we should wear the Super Puff Shorty with the Tna Butter Legging, Everyday T-Shirt and Solomon shoes! This is exactly what we expected it to predict, as the dataset we manually created had that item combination manually inserted most frequently.

## Overall, our model based off of Tiktok data and previous sales analytics has demonstrated an 82% accuracy in correctly predicting which garment can complete an outfit based on other outfit pieces. It has styled four awesome outfits pre-selected by the hacking team successfully, and can be used to power Aritzia's SmartStyle feature!